In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration ---
# Set the style for our plots to make them look nice
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (18, 8)
plt.rcParams["font.size"] = 12

# Define the path to your analysis file
# The filename is the one you provided.
file_path = "outputs/analysis_Phi-4-reasoning_20250621_123636.json"

print(f"Ready to analyze file: {file_path}")

In [ ]:
# Load the JSON data into a pandas DataFrame for easy manipulation

try:
    with open(file_path, "r") as f:
        data = json.load(f)
    
    # Convert the list of dictionaries directly into a DataFrame
    df = pd.DataFrame(data)
    avg_variance = df['variance'].mean()
    
    print("File loaded successfully!")
    print(f"Total number of generated tokens: {len(df)}")
    print(f"Average variance: {avg_variance:.2e}")
    
    
    # Display the first few rows to verify the structure
    print("\nData Head:")
    display(df.head())
    
    # Display summary information about the DataFrame
    print("\nData Info:")
    df.info()

except FileNotFoundError:
    print(f"ERROR: The file was not found at {file_path}")
    print("Please make sure the filename and path are correct.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(14, 6))
    plt.plot(
        df.index,
        df['sentropy'],
        label='Sentropy',
        color='blue',
        linewidth=0.5  # Make the line thin
    )
    plt.yscale('log')
    plt.gca().invert_yaxis()  # Flip the y-axis
    plt.xlabel('Token Step')
    plt.ylabel('Sentropy (log scale)')
    plt.title('Token Probability Sentropy per Generation Step')
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    
    # Highlight max and min variance points
    max_idx = df['sentropy'].idxmax()
    min_idx = df['sentropy'].idxmin()
    plt.scatter([max_idx], [df.loc[max_idx, 'sentropy']], color='red', label='Max')
    plt.scatter([min_idx], [df.loc[min_idx, 'sentropy']], color='green', label='Min')
    plt.legend()
    plt.savefig("sentropy_plot.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("DataFrame 'df' not available.")

In [ ]:
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(1400, 6))
    plt.plot(df.index, df['variance'], label='Variance', color='blue')
    plt.yscale('log')
    plt.xlabel('Token Step')
    plt.ylabel('Variance (log scale)')
    plt.title('Token Probability Variance per Generation Step')
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    
    # Highlight max and min variance points
    max_idx = df['variance'].idxmax()
    min_idx = df['variance'].idxmin()
    plt.scatter([max_idx], [df.loc[max_idx, 'variance']], color='red', label='Max')
    plt.scatter([min_idx], [df.loc[min_idx, 'variance']], color='green', label='Min')
    plt.legend()
    plt.show()
else:
    print("DataFrame 'df' not available.")

In [ ]:
# Show the 1st, 5th, and 40th percentile (lowest) variance values in scientific notation

if 'df' in locals() and not df.empty:
    percentiles = [0.01, 0.05, 0.40]
    percentile_labels = ['1st', '5th', '40th']
    values = df['sentropy'].quantile(percentiles)

    
    print("Entropy percentiles (lowest):")
    for label, val in zip(percentile_labels, values):
        print(f"{label} percentile: {val:.2e}")
else:
    print("DataFrame 'df' not available. Please run the previous cell successfully.")

In [ ]:
import numpy as np

if 'df' in locals() and not df.empty:
    plt.figure(figsize=(12, 6))
    sns.histplot(df['variance'], bins=50, kde=True)
    plt.xscale('log')
    plt.xlabel('Variance (log scale)')
    plt.ylabel('Token Count')
    plt.title('Distribution of Token Probability Variance')
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    
    # Add more ticks on the log x-axis
    min_var = df['variance'].min()
    max_var = df['variance'].max()
    ticks = np.logspace(np.log10(min_var), np.log10(max_var), num=10)
    plt.xticks(ticks, [f"{t:.1e}" for t in ticks])
    
    # Add median line
    median_var = df['variance'].median()
    plt.axvline(median_var, color='red', linestyle='--', label=f"Median: {median_var:.2e}")
    plt.legend()
    plt.show()
else:
    print("DataFrame 'df' not available.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

if 'df' in locals() and not df.empty:
    plt.figure(figsize=(12, 6))
    sns.histplot(df['sentropy'], bins=50, kde=True)
    # plt.xscale('log')  # <-- Removed for linear scale
    plt.xlabel('Sentropy')
    plt.ylabel('Token Count')
    plt.title('Distribution of Token Probability Sentropy')
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    
    # Use default linear ticks, or set manually if you want:
    # min_var = df['entropy'].min()
    # max_var = df['entropy'].max()
    # ticks = np.linspace(min_var, max_var, num=10)
    # plt.xticks(ticks, [f"{t:.2f}" for t in ticks])
    
    # Add median line
    median_var = df['sentropy'].median()
    plt.axvline(median_var, color='red', linestyle='--', label=f"Median: {median_var:.2f}")
    plt.legend()
    plt.savefig("sentropy_plot2.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("DataFrame 'df' not available.")

In [ ]:
import matplotlib.pyplot as plt

# Ask the user for the window size k
k = int(input("Enter window size k: "))

if 'df' in locals() and not df.empty and 'sentropy' in df.columns:
    # Compute rolling mean (moving average) with window size k
    rolling_sentropy = df['sentropy'].rolling(window=k).mean()

    plt.figure(figsize=(14, 6))
    plt.plot(df.index, rolling_sentropy, label=f'Rolling mean sentropy (window={k})', color='purple')
    plt.xlabel('Token Step')
    plt.ylabel('Mean Sentropy')
    plt.title(f'Rolling Mean Sentropy of Probability Distribution (k={k})')
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.legend()
    plt.show()
else:
    print("DataFrame 'df' not available or missing 'sentropy' column.")


In [ ]:
# --- Show the top 50 tokens by model probability at the step where a specified token was generated ---

token_of_interest = input("Enter part or all of the token to inspect: ").strip()

matches = df[df['chosen_token'].str.contains(token_of_interest, regex=False)]
if matches.empty:
    print(f"No generated token contains: '{token_of_interest}'")
else:
    print(f"Found {len(matches)} matches for '{token_of_interest}':")
    for i, (_, row) in enumerate(matches.iterrows(), 1):
        print(f"\nOccurrence {i}: Step {row['step']} (chosen_token: {row['chosen_token']!r}, sentropy={row['sentropy']:.2e}, variance={row['variance']:.2e})")
        top_preds = pd.DataFrame(row['top_k_predictions'])
        display(top_preds.sort_values('probability', ascending=False).reset_index(drop=True))